In [174]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'
TRAJECTORY_NAME = 'vabadussild_novatel'
# TRAJECTORY_NAME = 'vabadusepst_novatel'
# TRAJECTORY_NAME = 'oldtown_novatel'

model_type = ModelType.NN
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()


Trial 254 Complete [00h 00m 01s]
val_loss: 0.19507573544979095

Best val_loss So Far: 0.04220747947692871
Total elapsed time: 00h 04m 27s
INFO:tensorflow:Oracle triggered exit


Epoch 1/200
38/38 [==============================] - 1s 8ms/step - loss: 0.0384 - val_loss: 0.0455 - lr: 0.0010
Epoch 2/200
38/38 [==============================] - 0s 6ms/step - loss: 0.0181 - val_loss: 0.0447 - lr: 0.0010
Epoch 3/200
38/38 [==============================] - 0s 8ms/step - loss: 0.0123 - val_loss: 0.0456 - lr: 0.0010
Epoch 4/200
38/38 [==============================] - 0s 7ms/step - loss: 0.0092 - val_loss: 0.0438 - lr: 0.0010
Epoch 5/200
38/38 [==============================] - 0s 8ms/step - loss: 0.0068 - val_loss: 0.0443 - lr: 0.0010
Epoch 6/200
38/38 [==============================] - 0s 8ms/step - loss: 0.0040 - val_loss: 0.0440 - lr: 0.0010
Epoch 7/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0044 - val_loss: 0.0438 - lr: 0.0010
Epoch 8/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0045 - val_loss: 0.0442 - lr: 0.0010
Epoch 9/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0031 - val_loss: 0.0432 - lr:

In [176]:
processor.fit_trajectories()


131/131 [==============================] - 0s 2ms/step


In [177]:
processor.save_trajectories()


In [178]:
ape_fig = processor.create_ape_plot_all().show()


In [182]:
processor.create_ape_plot().show()


Min APE: 0.03066323549558127
Max APE: 6.716382202937813


In [179]:
# fig_2d = processor.create_2d_plot_utm()
# fig_2d.show()


In [180]:
# fig_slam = processor.create_2d_plot_slam()
# fig_slam.show()


In [181]:
# fig_map = processor.create_map_plot()
# fig_map.show()
